# Example: Monitoring Reforestation Projects
This notebook demonstrates how to work with **VerdeSat** services on a real-world dataset. The GeoJSON below represents three small reforestation projects in Chiapas, Mexico. We'll compute NDVI time series, land-cover metrics, a B-Score, and validate using citizen‑science observations.

In [ ]:
geojson_path = "examples/reforestation_plots.geojson"

In [ ]:
from verdesat.geo.aoi import AOI
from verdesat.ingestion.earthengine_ingestor import EarthEngineIngestor
from verdesat.ingestion.sensorspec import SensorSpec
from verdesat.analytics.timeseries import TimeSeries
from verdesat.services.landcover import LandcoverService
from verdesat.biodiv.metrics import MetricEngine
from verdesat.biodiv.bscore import BScoreCalculator
from verdesat.biodiv.gbif_validator import OccurrenceService, plot_score_vs_density


In [ ]:
aois = AOI.from_geojson(geojson_path, id_col='ID2')
for aoi in aois:
    print(aoi.static_props.get('id'), aoi.geometry.area)

In [ ]:
from verdesat.services.timeseries import download_timeseries

ts_df = download_timeseries(
    geojson=geojson_path,
    collection='NASA/HLS/HLSL30/v002',
    start='2019-01-01',
    end='2024-12-31',
    scale=30,
    chunk_freq='ME',
    agg='D',
    output='examples/reforestation_ts.csv',
)
ts_df.head()

In [ ]:
import ee
from ee import Reducer
sensor = SensorSpec.from_collection_id('NASA/HLS/HLSL30/v002')
base_ic = ee.ImageCollection('NASA/HLS/HLSL30/v002').filterDate('2022-01-01', '2022-12-31')
ndvi_coll = base_ic.map(lambda img: sensor.compute_index(img, 'ndvi'))
composites = AnalyticsEngine.build_composites(
    base_ic=ndvi_coll,
    period='ME',
    reducer=Reducer.mean(),
    start='2022-01-01',
    end='2022-12-31',
    bands=['NDVI'],
    scale=30,
)
composites.size().getInfo()  # number of composites

In [ ]:
from verdesat.analytics.timeseries import TimeSeries
from verdesat.analytics.engine import AnalyticsEngine
from verdesat.analytics.stats import compute_summary_stats
from verdesat.visualization.visualizer import Visualizer
import pandas as pd, os

ts = TimeSeries.from_dataframe(ts_df, index='ndvi')
ts_monthly = ts.aggregate('ME').fill_gaps()
decomposed = ts_monthly.decompose(period=12)

os.makedirs('examples/decomp', exist_ok=True)
for pid, res in decomposed.items():
    pd.DataFrame({
        'date': res.observed.index,
        'observed': res.observed.values,
        'trend': res.trend.values,
        'seasonal': res.seasonal.values,
        'resid': res.resid.values,
    }).to_csv(f'examples/decomp/{pid}_decomposition.csv', index=False)

trend = AnalyticsEngine.compute_trend(ts_monthly)
stats = compute_summary_stats('examples/reforestation_ts.csv', decomp_dir='examples/decomp', period=12)
stats.to_dataframe()

In [ ]:
viz = Visualizer()
viz.plot_time_series(ts_monthly.df, 'mean_ndvi', 'examples/ndvi_timeseries.png', agg_freq='ME')
viz.plot_timeseries_html(ts_monthly.df, 'mean_ndvi', 'examples/ndvi_timeseries.html', agg_freq='ME')
for pid, res in decomposed.items():
    viz.plot_decomposition(res, f'examples/decomp/{pid}_decomp.png')
trend.to_dataframe()

In [ ]:
df_stats = stats.to_dataframe()
df_scores = compute_bscores(geojson_path, year=2021, output='examples/bscore.csv')
df = df_stats.merge(df_scores, left_on='Site ID', right_on='id').set_index('Site ID')
df[['bscore', "Sen's Slope (NDVI/yr)"]].plot.bar(figsize=(8,4))
df.head()

In [ ]:
# All plots have been written to the examples directory.

### What do the metrics tell us?

* **Mean NDVI** represents the average greenness of each plot over time.
* **Seasonal amplitude** captures the difference between peak and trough NDVI values, highlighting intra‑annual variability.
* **Sen's slope** estimates the yearly change in NDVI after removing seasonal effects.
* **Residual RMS** indicates how much short‑term noise remains after decomposition.

Together these metrics help compare the three reforestation plots and understand which ones are improving the fastest.